In [1]:
'''
A notebook which allows a pretrained detector to be applied to 
any audio file.
'''
%load_ext autoreload
%autoreload 2
%matplotlib inline

# General imports
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import yaml
import cPickle as pickle
from scipy.ndimage.interpolation import zoom

# Neural network imports
import nolearn
import lasagne

# Custom functions and classes
sys.path.append('../..')
from lib import train_helpers

# Wav file loading and manipulation
import librosa
from librosa.feature import melspectrogram

Using gpu device 0: GeForce GTX 770 (CNMeM is disabled, cuDNN 5110)
/home/michael/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/home/michael/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


## Testing options

In [7]:
# You can provide here either:
#   A path to a single .wav file, 
# or
#   A path to a folder which contains one or more .wav files
#
# In this example code, we give a folder which contains a single .wav file.
# We could also give the complete path to this file - the result would be the same
# test_file_or_folder = '/home/michael/Dropbox/FairbrassFirmanetal_/models/example_test_data/'
# test_file_or_folder = '/media/michael/Engage/data/audio/alison_data/diversity_labelled_data/'
test_file_or_folder = '/media/michael/Elements1/Fieldwork_Data/2015/IG62XL/SM2+/'
test_file_or_folder = '/media/michael/Elements1/Fieldwork_Data/2015/E29RR/250515-010615/SM2+/'

# specify where the pretrained model is that we want to load
models_dir = '/home/michael/projects/alison_audio2/models/biotic/'

# Specify the names of the files we want to load in
# (Keep as they are to load in the model in the dropbox folder)
weights_path = os.path.join(models_dir, 'weights_99.pkl')
options_path = os.path.join(models_dir, 'network_opts.yaml')

In [8]:
# Loading the options for network architecture, spectrogram type etc
options = yaml.load(open(options_path))

# Checking the weights exist
assert os.path.exists(weights_path), 'Error - cannot find the weights file at ' + weights_path

## Finding the .wav files to test on

In [9]:
if os.path.isdir(test_file_or_folder):
    
    # User provided a folder - we must get the list of wav files
    wav_dir = test_file_or_folder
    filenames = [xx for xx in os.listdir(wav_dir) if xx.endswith('.wav')]
    assert len(filenames), 'Error - no files found!'

elif os.path.isfile(test_file_or_folder):
    
    # User provided a file - we must deal with the filename appropriately
    wav_dir = os.path.dirname(test_file_or_folder)
    filenames = [os.path.basename(test_file_or_folder)]

else:
    raise Exception("No such file or folder: ", test_file_or_folder)
    
print "We will load %d file(s) from the folder:" % len(filenames)
print wav_dir
print 
print "Files are:"
for filename in filenames:
    print "  -", filename

Exception: ('No such file or folder: ', '/media/michael/Elements1/Fieldwork_Data/2015/E29RR/250515-010615/SM2+/')

## Setting up network

In [6]:
# Create the layers of the neural network, with the same options we used in training
net_options = {xx: options[xx] for xx in train_helpers.net_params}
network = train_helpers.create_net(**net_options)

# Create an object which will iterate over the test spectrograms appropriately
test_sampler = train_helpers.SpecSampler(
    4, options['HWW'], False, options['LEARN_LOG'], randomise=0, seed=10)

# Create a nolearn object to contain the network and push data through it
net = nolearn.lasagne.NeuralNet(
    layers=network['prob'], update=lasagne.updates.adam, batch_iterator_test=test_sampler)

# Initialise the network and load in the pretrained parameters
net.initialize()
net.load_params_from(weights_path)

/home/michael/anaconda/lib/python2.7/site-packages/nolearn/lasagne/base.py:430: UserWarning: The Param class is deprecated. Replace Param(default=N) by theano.In(value=N)
  for input_layer in input_layers]
/home/michael/anaconda/lib/python2.7/site-packages/nolearn/lasagne/base.py:431: UserWarning: The Param class is deprecated. Replace Param(default=N) by theano.In(value=N)
  inputs = X_inputs + [theano.Param(y_batch, name="y")]


## Generating spectrogram(s)

In [7]:
# We will store spectrograms in a dictionary, where keys are the filenames
specs = {}  

# Loop over each file we want to predict for
for filename in filenames:
    
    if filename in specs:
        continue

    # Read in the wav file
    wav, sample_rate = librosa.load(os.path.join(wav_dir, filename), 22050)

    # Compute the spectrogram
    spec = melspectrogram(wav, sr=sample_rate, n_fft=options['N_FFT'], 
                          hop_length=options['HOP_LENGTH'], n_mels=options['N_MELS'])

    # Do log conversion:
    spec = np.log(options['A'] + options['B'] * spec)
    spec -= np.median(spec, axis=1, keepdims=True)

    # Add to the dictionary
    specs[filename] = spec.astype(np.float32)

## Applying classifier

In [8]:
# Push this list through the network
# test_sampler, defined above, does the hard work here.
from tqdm import tqdm
preds = {}

for filename in tqdm(filenames):
    preds[filename] = net.predict_proba([specs[filename]])

## Saving results to disk

In [9]:
savedir = '/home/michael/Dropbox/engage/FairbrassFirmanetal_/data/predictions/ours_on_all_data/'
for fname, pred in preds.iteritems():
    with open(savedir + fname.replace('.wav', '_' + options['CLASSNAME'] + '.pkl'), 'w') as f:
        pickle.dump(pred, f, -1)